In [1]:
%load_ext autoreload
%autoreload 2
!export PYTHONPATH=$PYTHONPATH:/Users/arvindagarwal/Desktop/cmu/courses/ml/lda2vec-pytorch/utils


In [2]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models
import pickle
import pandas as pd
from pprint import pprint
import csv 

import sys
sys.path.insert(0,'.')
sys.path.insert(0,'/Users/arvindagarwal/Desktop/cmu/courses/ml/lda2vec-pytorch/utils')
# from utils import preprocess, get_windows
# from utils.preprocess import preprocess
from preprocess_mod import *
from get_windows_mod import *

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['via'])
import re


paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [3]:
!ls ..

20newsgroups                   semeval2016-task6-domaincorpus
LICENSE                        stance
README.md                      utils
loss.png


In [4]:
MIN_COUNTS = 20
MAX_COUNTS = 1800
# words with count < MIN_COUNTS
# and count > MAX_COUNTS
# will be removed

MIN_LENGTH = 15
# minimum document length 
# (number of words)
# after preprocessing

# half the size of the context around a word
HALF_WINDOW_SIZE = 5
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

# Load NLP model

In [5]:
nlp = spacy.load('en')

# Load dataset

In [6]:
with open('slangs.csv', mode='r') as infile:
    reader = csv.reader(infile) 
    slang_words = {rows[0]:rows[1] for rows in reader}

In [7]:
def clean_tweets(sent):
    sent = str(sent)
    
    # Substitute Slangs
    for word in sent.split(" "):
        if word in slang_words.keys():
            sent = re.sub(word, slang_words[word], sent)
            
    # Remove new line characters
    sent = re.sub('\s+', ' ', sent)

    # Remove http:// links
    sent = re.sub('http:\/\/.*','', sent)

    # Remove https:// links
    sent = re.sub('https:\/\/.*','', sent)
    
    # Remove distracting single quotes
    sent = re.sub("\'", "", sent)

    # Remove distracting single quotes
    sent = re.sub("\"", "", sent)

    # Remove hashtags
    sent = re.sub("\#", "", sent)
    
    return sent

In [8]:
# dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
# docs = dataset['data']

In [ ]:
PATH_LABELLED_DATA_TRUMP = "../semeval2016-task6-domaincorpus/data-all-annotations/testdata-taskB-all-annotations.txt"
PATH_UNLABELLED_DATA_TRUMP = "./../semeval2016-task6-domaincorpus/downloaded_Donald_Trump.txt"

In [ ]:
X_train = pd.read_pickle('X_train.pkl')
X_test = pd.read_pickle('X_test.pkl')
y_train = pd.read_pickle('y_train.pkl')
y_test = pd.read_pickle('y_test.pkl')

data_labelled_train = pd.concat([X_train, y_train], ignore_index=True, axis=1)
data_labelled_train.columns = ['Tweet', 'Stance']
data_labelled_test = pd.concat([X_test, y_test], ignore_index=True, axis=1)
data_labelled_test.columns = ['Tweet', 'Stance']

In [ ]:
data_unlabelled = pd.read_csv(PATH_UNLABELLED_DATA_TRUMP, sep='\t', lineterminator='\n', encoding ='latin1', names = ["ID", "Tweet"])
data_unlabelled = data_unlabelled.where(data_unlabelled.Tweet != 'Not Available')
data_unlabelled.dropna(how='any', inplace=True)
data_unlabelled['Tweet'] = data_unlabelled['Tweet'].apply(lambda x: x[1:])
data_unlabelled['Tweet'] = data_unlabelled['Tweet'].apply(clean_tweets)

In [ ]:
combined_data = pd.concat([data_labelled_test, data_labelled_train, data_unlabelled], ignore_index=True)

/Users/arvindagarwal/anaconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [ ]:
combined_data.count()

ID        53616
Stance      897
Tweet     54513
dtype: int64

In [ ]:
combined_data.head(10)

,ID,Stance,Tweet
0,NaN,AGAINST,We need a man that will do the tough negotiati...
1,NaN,AGAINST,"WE Love Mexicans,WE Respect your Work Ethic,Yo..."
2,NaN,NONE,ExtremistProgressives are so Focused on their ...
3,NaN,AGAINST,@BraveConWarrior stories like this make think ...
4,NaN,FAVOR,GOP candidate predictions? America decides to ...
5,NaN,NONE,"...And of course, God bless America!! tcot tea..."
6,NaN,FAVOR,Donald Trump for president? I can dig it. SemST
7,NaN,NONE,@ericbolling :). Sorry for the kidding but had...
8,NaN,NONE,Muslim community chips in to help rebuild burn...
9,NaN,FAVOR,This country needs a President to take control...


In [ ]:
df = combined_data['Tweet']

In [ ]:
data = df.values.tolist()
docs = data
pprint(data[:5])

['We need a man that will do the tough negotiations, say what needs to be said '
 'and forget the words politically correct. thanks SemST',
 'WE Love Mexicans,WE Respect your Work Ethic,Your Love of Family,Your '
 'Loyalty,Your FOOD &Your Love Of God! Lets ALL Get JOBS,VOTE SemST',
 'ExtremistProgressives are so Focused on their Agenda that they believe a '
 'Inanimate FlagKills but MultitimeDeporteesDont SemST',
 '@BraveConWarrior stories like this make think PATRIOTS are at the end of the '
 'string. all kinds of shit is going to hit fan SemST',
 'GOP candidate predictions? America decides to play the Trump card and go '
 'with the man with the plan. SemST']


In [ ]:
# number of documents
len(docs)

54513

In [ ]:
# store an index with a document
docs = [(i, doc) for i, doc in enumerate(docs)]

# Preprocess dataset and create windows

In [ ]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)

 82%|████████▏ | 44597/54513 [08:40<02:09, 76.56it/s] 

In [ ]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [ ]:
data = []
# new ids are created here
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')

In [ ]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

In [ ]:
# number of windows (equals to the total number of tokens)
data.shape[0]

# Get unigram distribution

In [ ]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

# Prepare word vectors

In [ ]:
# %%time
vocab_size = len(decoder)
embedding_dim = 50

# train a skip-gram word2vec model
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
model = models.Word2Vec(texts, size=embedding_dim, window=5, workers=4, sg=1, negative=15, iter=70)
model.init_sims(replace=True)

word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')
for i in decoder:
    word_vectors[i] = model.wv[str(i)]

In [ ]:
# number of unique words
vocab_size

# Prepare initialization for document weights

In [ ]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
# %%time
n_topics = 25
lda = models.LdaModel(corpus, alpha=0.9, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

In [ ]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

In [ ]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob

# Save data

In [ ]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)